# producing and validating $p(z | photometry)$ for ELAsTiCC

_Alex Malz (~~GCCL@RUB~~LINCC@CMU)_

The goal here is to generate mock photo-$z$ posteriors for host galaxies. 
These ones will be Gaussian -- self-consistent, but not realistically complex.

In [ ]:
import bisect
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
random.seed = 42
import scipy.stats as sps
import subprocess
import sys
eps = sys.float_info.epsilon

## the hostlibs

In [ ]:
hl_heads = {'SNIa': None,
            'SNII': None, 
            'SNIbc': None, 
            'UNMATCHED_KN_SHIFT': None,
            'UNMATCHED_COSMODC2': None}

let's pick one hostlib for now.

In [ ]:
pick_one = 4
which_hl = list(hl_heads.keys())[pick_one]
in_path = '/global/cfs/cdirs/lsst/groups/TD/SN/SNANA/SURVEYS/LSST/ROOT/PLASTICC_DEV/HOSTLIB/'+which_hl+'_GHOST.HOSTLIB.gz'
hl_head = int(subprocess.check_output(f"zcat {in_path} | cat -n | sed -n '/VARNAMES/ {{ p; q }}'  | awk '{{print $1-1}}'", shell=True))

this is somewhat slow because the files are ~GB

In [ ]:
df = pd.read_csv(in_path, skiprows=hl_head, delimiter=' ', header=1)
# df.set_index('GALID')
nhost = len(df)
# nhost = 100
# df = df[:nhost]

## forecasting-level photo-z PDFs

using the values from the DESC SRD

### first create likelihood $p(z_{obs} | z_{true})$

using values from [DESC SRD](https://arxiv.org/abs/1809.01669)

In [ ]:
sigma = 0.02

In [ ]:
true_locs = df['ZTRUE'].values.reshape((nhost, 1))

stupid transform because `scipy.stats.truncnorm` assumes input endpoints are given for standard normal and _then_ rescales them to provided mean and standard deviation

In [ ]:
lik_min = (0. - true_locs) / (sigma * (1. + true_locs))
lik_max = (3. - true_locs) / (sigma * (1. + true_locs))
# lik_min = (0-true_locs) / (sigma * (1. + true_locs))
# lik_max = (3-true_locs) / (sigma * (1. + true_locs))

In [ ]:
likelihood = sps.truncnorm(a=lik_min, b=lik_max, loc=true_locs, scale=sigma*(1.+true_locs))
# pseudo_likelihood = sps.truncnorm(a=lik_min, b=lik_max, loc=0., scale=1.)

### then draw point estimate $z_{obs} \sim p(z_{obs} | z_{true})$

warning: this is slow because the endpoints are different -- stupid scipy defaults!

In [ ]:
# pseudo_obs = pseudo_likelihood.rvs(random_state=42)
# obs_locs = pseudo_obs * (sigma * (1. + true_locs)) + true_locs
obs_locs = likelihood.rvs(random_state=42)

check that these make sense

In [ ]:
plt.scatter(true_locs, obs_locs, s=0.1, alpha=0.1)
plt.xlabel('$z_{true}$')
plt.ylabel('$z_{obs}$')
plt.plot([0., 3.], [0., 3.], c='k')

### then make posterior $p(z_{true} | z_{obs})$

this is what photo-z PDF estimators actually produce

sadly have to do the stupid scipy rescaling again

In [ ]:
pos_min = (0. - obs_locs) / (sigma * (1. + obs_locs))
pos_max = (3. - obs_locs) / (sigma * (1. + obs_locs))

In [ ]:
posterior = sps.truncnorm(a=pos_min, b=pos_max, loc=obs_locs, scale=sigma*(1.+obs_locs))

plot a few examples

In [ ]:
zgrid = np.logspace(-3., np.log10(3.), 300)
to_plot = random.sample(range(nhost), 10)

In [ ]:
fig, ax = plt.subplots(10, 1, figsize=(5, 20))
for i, galind in enumerate(to_plot):
    example = sps.truncnorm(a=pos_min[galind][0], b=pos_max[galind][0], 
                            loc=obs_locs[galind][0], scale=sigma*(1+obs_locs[galind][0])).pdf(zgrid)
    ax[i].plot(zgrid, example)
    ax[i].vlines(true_locs[galind][0], 0, max(example), color='k', linewidth=.75)
    ax[i].vlines(obs_locs[galind][0], 0, max(example), color='r', linewidth=.75)
    ax[i].set_ylim(0, 15)
    ax[i].text(2, 10, str(df['GALID'].loc[i]))
    ax[i].set_ylabel('$p(z)$')
    if i == len(to_plot)-1:
        ax[i].set_xlabel('$z$')
    else:
        ax[i].set_xticklabels([])
fig.subplots_adjust(wspace=0, hspace=0)

## evaluate quantiles

using regular grid for now but can be optimized depending on science case sensitive to outliers vs. degeneracies vs. general spread

In [ ]:
quants = np.linspace(0., 1., 11)
quants[0] += eps
quants[-1] -= eps

In [ ]:
quantlabs = ['ZPHOT_Q000', 'ZPHOT_Q010', 'ZPHOT_Q020', 'ZPHOT_Q030', 'ZPHOT_Q040', 'ZPHOT_Q050', 'ZPHOT_Q060', 'ZPHOT_Q070', 'ZPHOT_Q080', 'ZPHOT_Q090', 'ZPHOT_Q100']

this step is slow!

In [ ]:
df[quantlabs] = posterior.ppf(quants)

plot one just to see what the quantiles look like for a given PDF

In [ ]:
plot_one = random.sample(range(nhost), 1)
plt.plot(zgrid, sps.truncnorm(a=pos_min[plot_one][0], b=pos_max[plot_one][0], 
                              loc=obs_locs[plot_one][0], scale=sigma*(1+obs_locs[plot_one][0])).pdf(zgrid))
plt.vlines(df[quantlabs].loc[plot_one], -1, 1, linestyle='--', color='k')
# plt.xlim(obs_locs[plot_one][0]-5*sigma*(1+obs_locs[plot_one][0]), 
#          obs_locs[plot_one][0]+5*sigma*(1+obs_locs[plot_one][0]))
plt.xlim(df['ZPHOT_Q000'].loc[plot_one].values[0]-0.01, df['ZPHOT_Q100'].loc[plot_one].values[0]+0.01)
plt.text(obs_locs[plot_one][0], 2, str(df['GALID'].loc[plot_one[0]]))
plt.xlabel('$z$')
plt.ylabel('$p(z)$')

plot histogram of first and last quantile, since these aren't really well-defined

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].hist(df['ZPHOT_Q000'], bins=300)
ax[0].set_xlabel('$z_{0\%}$')
ax[1].hist(df['ZPHOT_Q100'], bins=300)
ax[1].set_xlabel('$z_{100\%}$')

also save $p(z_{median})$ and [inter-quartile range](https://en.wikipedia.org/wiki/Interquartile_range)

unfortunately these are quite slow now!

In [ ]:
df['IQR_ZPHOT'] = posterior.ppf(0.75) - posterior.ppf(0.25)

In [ ]:
df['P_ZPHOT'] = posterior.pdf(posterior.median())

save a file

In [ ]:
out_path = '/global/cfs/cdirs/lsst/groups/TD/SN/SNANA/SURVEYS/LSST/ROOT/PLASTICC_DEV/HOSTLIB/zquants/'+which_hl+'_dummy_pz.csv'
df.to_csv(out_path, index=False, sep=' ')

## reconstructing a pdf from quantiles

read from a generic hostlib file

In [ ]:
df = pd.read_csv(out_path, delimiter=' ', header=0)

pick one for demonstration

In [ ]:
plot_one = random.sample(range(nhost), 1)[0]
zq_vals = df[quantlabs].loc[plot_one].values

This is the reconstruction algorithm from [ye olde qp](https://github.com/aimalz/qp/blob/master/qp/pdf.py#L554).

Note: The aforementioned code exhibits unexpected behavior in Python 3; 
you must (unfortunately) run it in Python 2 for consistency with [Malz & Marshall+ 2017](http://stacks.iop.org/1538-3881/156/i=1/a=35).

Another note: You can save yourself one float by replacing the redshifts where $CDF=0$ and $CDF=1$ with $p(z_{q})$ for any of the saved quantiles $q$, at the cost of some inaccuracy in the tails.

In [ ]:
# derivative = (quants[1:] - quants[:-1]) / (zq_vals[1:] - zq_vals[:-1])
# derivative = np.insert(derivative, 0, 0.)
# derivative = np.append(derivative, 0.)

# def pdf_inside(zgrid):
#     pdf = np.zeros_like(zgrid)
#     for n in range(len(zgrid)):
#         ind = bisect.bisect_left(zq_vals, zgrid[n])
#         pdf[n] = derivative[ind]
#     return(pdf)

q = quants
z = zq_vals

derivative = (q[1:] - q[:-1]) / (z[1:] - z[:-1])
derivative = np.insert(derivative, 0, eps)
derivative = np.append(derivative, eps)
def pdf_inside(xf):
    nx = len(xf)
    yf = np.ones(nx) * eps
    for n in range(nx):
        i = bisect.bisect_left(z, xf[n])
        yf[n] = derivative[i]
    return(yf)

eval_pdf = pdf_inside(zgrid)

show difference between original and reconstruction

In [ ]:
plt.plot(zgrid, sps.truncnorm(a=pos_min[plot_one][0], b=pos_max[plot_one][0], 
                              loc=obs_locs[plot_one][0], scale=sigma*(1+obs_locs[plot_one][0])).pdf(zgrid),
                              label='original PDF')
plt.vlines(df[quantlabs].loc[plot_one], -1, 1, linestyle='--')
plt.xlim(zq_vals[0] - 0.01, zq_vals[-1] + 0.01)
plt.plot(zgrid, eval_pdf, label='reconstructed from '+str(len(quants))+' quantiles')
plt.text(obs_locs[plot_one][0], 2, str(df['GALID'].loc[plot_one]))
plt.legend(loc='upper right')
plt.xlabel('$z$')
plt.ylabel('$p(z)$')